In [3]:
##Set up folders, templates, and helpers
# Bootstrap the project structure (run once) ---

import os, json, re, datetime
from pathlib import Path

# Project directories
BASE_DIR   = Path.cwd() / "personal_blog"
ARTICLES   = BASE_DIR / "articles"
TEMPLATES  = BASE_DIR / "templates"
STATIC_DIR = BASE_DIR / "static"

for d in (BASE_DIR, ARTICLES, TEMPLATES, STATIC_DIR):
    d.mkdir(parents=True, exist_ok=True)

# -------------------
# Minimal CSS
# -------------------
css = """
:root {
  --bg:#0b0f14; --card:#121821; --muted:#8aa0b2; --ink:#e7eef5; --accent:#6ea8fe; --danger:#ff6b6b; --ok:#4dd4ac;
}
*{box-sizing:border-box}
body{margin:0; font-family:system-ui, -apple-system, Segoe UI, Roboto, Inter, Arial, sans-serif; background:var(--bg); color:var(--ink)}
a{color:var(--accent); text-decoration:none}
a:hover{text-decoration:underline}
.container{max-width:900px; margin:0 auto; padding:24px}
.card{background:var(--card); border-radius:16px; padding:20px; box-shadow:0 4px 20px rgba(0,0,0,.35)}
.grid{display:grid; gap:16px}
.btn{display:inline-block; padding:.6rem 1rem; border-radius:12px; border:1px solid #2a3442}
.btn.primary{background:var(--accent); border-color:var(--accent); color:#051225}
.btn.warn{background:transparent; color:var(--danger); border-color:#3c1f24}
.btn.ghost{background:transparent; color:var(--muted)}
input[type="text"], input[type="date"], textarea{
  width:100%; padding:.7rem .9rem; border-radius:12px; border:1px solid #2a3442;
  background:#0c131b; color:var(--ink)
}
label{display:block; font-size:.92rem; color:var(--muted); margin-bottom:.4rem}
.field{margin-bottom:1rem}
.header{display:flex; align-items:center; justify-content:space-between; margin-bottom:16px}
.nav a{margin-right:14px}
.badge{font-size:.8rem; color:var(--muted)}
h1,h2,h3{margin:0 0 .6rem 0}
ul.reset{list-style:none; padding:0; margin:0}
.article-item{padding:14px; border:1px solid #212a36; border-radius:14px}
hr{border:none; border-top:1px solid #1d2631; margin:20px 0}
.flash{padding:.7rem 1rem; border-radius:10px; background:#0f151e; border:1px solid #203043; color:#b9e3ff}
.form-actions{display:flex; gap:10px; align-items:center}
footer{margin-top:28px; color:var(--muted); font-size:.9rem}
.empty{color:var(--muted)}
"""

(STATIC_DIR / "styles.css").write_text(css, encoding="utf-8")

# -------------------
# Jinja2 templates
# -------------------
base_html = """<!doctype html>
<html lang="en">
  <head>
    <meta charset="utf-8">
    <meta name="viewport" content="width=device-width,initial-scale=1">
    <title>{{ title or "Personal Blog" }}</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='styles.css') }}">
  </head>
  <body>
    <div class="container">
      <div class="header">
        <div class="nav">
          <a href="{{ url_for('home') }}">Home</a>
          {% if session.get('admin') %}
            <a href="{{ url_for('dashboard') }}">Dashboard</a>
            <a href="{{ url_for('logout') }}" class="btn ghost">Logout</a>
          {% else %}
            <a href="{{ url_for('login') }}" class="btn ghost">Login</a>
          {% endif %}
        </div>
        <div class="badge">Personal Blog</div>
      </div>
      {% with messages = get_flashed_messages() %}
        {% if messages %}
          <div class="flash">
            {% for m in messages %}{{ m }}{% if not loop.last %}<br>{% endif %}{% endfor %}
          </div>
        {% endif %}
      {% endwith %}
      <div class="card">
        {% block content %}{% endblock %}
      </div>
      <footer>© {{ now.year }} — A tiny Flask blog • No JS, just vibes</footer>
    </div>
  </body>
</html>"""

home_html = """{% extends "base.html" %}
{% block content %}
  <h1>Articles</h1>
  <p class="badge">Latest posts</p>
  <hr>
  {% if articles %}
    <div class="grid">
      {% for a in articles %}
        <div class="article-item">
          <h3><a href="{{ url_for('article', slug=a.slug) }}">{{ a.title }}</a></h3>
          <div class="badge">Published: {{ a.date }}</div>
        </div>
      {% endfor %}
    </div>
  {% else %}
    <p class="empty">No articles yet.</p>
  {% endif %}
{% endblock %}
"""

article_html = """{% extends "base.html" %}
{% block content %}
  <a href="{{ url_for('home') }}" class="btn ghost">← Back</a>
  <h1 style="margin-top:10px">{{ article.title }}</h1>
  <div class="badge">Published: {{ article.date }}</div>
  <hr>
  <div style="white-space:pre-wrap; line-height:1.6">{{ article.content }}</div>
{% endblock %}
"""

login_html = """{% extends "base.html" %}
{% block content %}
  <h1>Admin Login</h1>
  <p class="badge">Restricted area</p>
  <hr>
  <form method="post">
    <div class="field">
      <label>Username</label>
      <input type="text" name="username" required>
    </div>
    <div class="field">
      <label>Password</label>
      <input type="text" name="password" required>
    </div>
    <div class="form-actions">
      <button class="btn primary" type="submit">Login</button>
    </div>
  </form>
{% endblock %}
"""

dashboard_html = """{% extends "base.html" %}
{% block content %}
  <div class="header" style="margin:0 0 8px 0">
    <h1 style="margin:0">Dashboard</h1>
    <a class="btn primary" href="{{ url_for('add_article') }}">+ New Article</a>
  </div>
  <p class="badge">Manage your content</p>
  <hr>
  {% if articles %}
    <ul class="reset grid">
      {% for a in articles %}
        <li class="article-item">
          <div style="display:flex; justify-content:space-between; gap:10px; align-items:center; flex-wrap:wrap">
            <div>
              <strong>{{ a.title }}</strong><br>
              <span class="badge">{{ a.date }} • slug: {{ a.slug }}</span>
            </div>
            <div class="form-actions">
              <a class="btn" href="{{ url_for('edit_article', slug=a.slug) }}">Edit</a>
              <form method="post" action="{{ url_for('delete_article', slug=a.slug) }}" onsubmit="return confirm('Delete this article?');">
                <button class="btn warn" type="submit">Delete</button>
              </form>
            </div>
          </div>
        </li>
      {% endfor %}
    </ul>
  {% else %}
    <p class="empty">No articles yet. Click “New Article”.</p>
  {% endif %}
{% endblock %}
"""

form_html = """{% extends "base.html" %}
{% block content %}
  <h1>{{ heading }}</h1>
  <p class="badge">{{ subheading }}</p>
  <hr>
  <form method="post">
    <div class="field">
      <label>Title</label>
      <input type="text" name="title" required value="{{ article.title if article else '' }}">
    </div>
    <div class="field">
      <label>Date of publication</label>
      <input type="date" name="date" required value="{{ article.date if article else '' }}">
    </div>
    <div class="field">
      <label>Content</label>
      <textarea name="content" rows="12" required>{{ article.content if article else '' }}</textarea>
    </div>
    <div class="form-actions">
      <button class="btn primary" type="submit">{{ cta }}</button>
      <a href="{{ url_for('dashboard') }}" class="btn ghost">Cancel</a>
    </div>
  </form>
{% endblock %}
"""

(TEMPLATES / "base.html").write_text(base_html, encoding="utf-8")
(TEMPLATES / "home.html").write_text(home_html, encoding="utf-8")
(TEMPLATES / "article.html").write_text(article_html, encoding="utf-8")
(TEMPLATES / "login.html").write_text(login_html, encoding="utf-8")
(TEMPLATES / "dashboard.html").write_text(dashboard_html, encoding="utf-8")
(TEMPLATES / "form.html").write_text(form_html, encoding="utf-8")

# -------------------
# Create a sample article so Home isn't empty
# -------------------
sample_path = ARTICLES / "hello-world.json"
if not sample_path.exists():
    sample = {
        "title":"Hello, World!",
        "date": datetime.date.today().isoformat(),
        "content": "This is your first post. Edit or delete it from the Dashboard.",
        "slug": "hello-world"
    }
    sample_path.write_text(json.dumps(sample, indent=2), encoding="utf-8")

print(f"Project scaffold created at: {BASE_DIR}")
print("Templates and a sample article are ready.")


Project scaffold created at: C:\Users\krish\Documents\Krishna\Personal Blog\personal_blog
Templates and a sample article are ready.


In [ ]:
##The Flask app (guest + admin)
# Flask app (run to start the server) ---

import os, json, datetime, re
from pathlib import Path
from flask import Flask, render_template, request, redirect, url_for, flash, session, abort, send_from_directory

BASE_DIR   = Path.cwd() / "personal_blog"
ARTICLES   = BASE_DIR / "articles"
TEMPLATES  = BASE_DIR / "templates"
STATIC_DIR = BASE_DIR / "static"

# --- Config (you can change these) ---
ADMIN_USERNAME = "admin"
ADMIN_PASSWORD = "password123"  # change me
SECRET_KEY     = "dev-secret-change-me"

# --- App ---
app = Flask(__name__, template_folder=str(TEMPLATES), static_folder=str(STATIC_DIR))
app.config["SECRET_KEY"] = SECRET_KEY

# -------- Storage helpers --------
def slugify(title: str) -> str:
    s = re.sub(r"[^a-zA-Z0-9\- ]+", "", title).strip().lower()
    s = re.sub(r"\s+", "-", s)
    s = re.sub(r"-+", "-", s)
    return s or "post"

def article_path(slug: str) -> Path:
    return ARTICLES / f"{slug}.json"

def load_article(slug: str):
    p = article_path(slug)
    if not p.exists():
        return None
    return json.loads(p.read_text(encoding="utf-8"))

def save_article(data: dict):
    # ensure slug exists and is unique
    slug = data.get("slug") or slugify(data["title"])
    i = 1
    base = slug
    # Only enforce uniqueness on create; if editing and slug unchanged, allow overwrite
    while i < 1000 and article_path(slug).exists() and json.loads(article_path(slug).read_text()).get("title") != data["title"]:
        slug = f"{base}-{i}"
        i += 1
    data["slug"] = slug
    article_path(slug).write_text(json.dumps(data, indent=2), encoding="utf-8")
    return slug

def delete_article_file(slug: str):
    p = article_path(slug)
    if p.exists():
        p.unlink()

def list_articles():
    items = []
    for f in ARTICLES.glob("*.json"):
        try:
            a = json.loads(f.read_text(encoding="utf-8"))
            items.append(a)
        except Exception:
            continue
    # sort by date desc, then title
    def parse_date(d):
        try:
            return datetime.date.fromisoformat(d)
        except Exception:
            return datetime.date.min
    items.sort(key=lambda x: (parse_date(x.get("date","1970-01-01")), x.get("title","")), reverse=True)
    return items

# -------- Auth helpers --------
from functools import wraps

def admin_required(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if not session.get("admin"):
            flash("Please log in as admin.")
            return redirect(url_for("login"))
        return fn(*args, **kwargs)
    return wrapper

# -------- Guest routes --------
@app.route("/")
def home():
    return render_template("home.html", articles=list_articles(), now=datetime.datetime.now(), title="Home")

@app.route("/article/<slug>")
def article(slug):
    a = load_article(slug)
    if not a:
        abort(404)
    return render_template("article.html", article=a, now=datetime.datetime.now(), title=a["title"])

# -------- Auth routes --------
@app.route("/admin/login", methods=["GET","POST"])
def login():
    if request.method == "POST":
        u = request.form.get("username","").strip()
        p = request.form.get("password","").strip()
        if u == ADMIN_USERNAME and p == ADMIN_PASSWORD:
            session["admin"] = True
            flash("Logged in.")
            return redirect(url_for("dashboard"))
        flash("Invalid credentials.")
    return render_template("login.html", now=datetime.datetime.now(), title="Login")

@app.route("/admin/logout")
def logout():
    session.clear()
    flash("Logged out.")
    return redirect(url_for("home"))

# -------- Admin routes --------
@app.route("/admin/dashboard")
@admin_required
def dashboard():
    return render_template("dashboard.html", articles=list_articles(), now=datetime.datetime.now(), title="Dashboard")

@app.route("/admin/add", methods=["GET","POST"])
@admin_required
def add_article():
    if request.method == "POST":
        title = request.form["title"].strip()
        date  = request.form["date"].strip()
        content = request.form["content"].strip()
        if not title or not date or not content:
            flash("All fields are required.")
            return redirect(url_for("add_article"))
        data = {"title": title, "date": date, "content": content}
        slug = save_article(data)
        flash("Article created.")
        return redirect(url_for("dashboard"))
    return render_template("form.html",
                           heading="Add Article",
                           subheading="Create a new post",
                           cta="Publish",
                           article=None,
                           now=datetime.datetime.now(),
                           title="Add Article")

@app.route("/admin/edit/<slug>", methods=["GET","POST"])
@admin_required
def edit_article(slug):
    a = load_article(slug)
    if not a:
        abort(404)
    if request.method == "POST":
        title = request.form["title"].strip()
        date  = request.form["date"].strip()
        content = request.form["content"].strip()
        if not title or not date or not content:
            flash("All fields are required.")
            return redirect(url_for("edit_article", slug=slug))
        # Preserve slug; allow title change without changing filename to keep URLs stable
        a.update({"title": title, "date": date, "content": content})
        article_path(slug).write_text(json.dumps(a, indent=2), encoding="utf-8")
        flash("Article updated.")
        return redirect(url_for("dashboard"))
    return render_template("form.html",
                           heading="Edit Article",
                           subheading=f"Editing “{a['title']}”",
                           cta="Save changes",
                           article=a,
                           now=datetime.datetime.now(),
                           title=f"Edit: {a['title']}")

@app.route("/admin/delete/<slug>", methods=["POST"])
@admin_required
def delete_article(slug):
    if not load_article(slug):
        flash("Article not found.")
        return redirect(url_for("dashboard"))
    delete_article_file(slug)
    flash("Article deleted.")
    return redirect(url_for("dashboard"))

# Convenience route to help you see where files live (optional)
@app.route("/_files/<path:filename>")
@admin_required
def files(filename):
    return send_from_directory(BASE_DIR, filename)

if __name__ == "__main__":
    # In Jupyter, use a non-blocking run if you prefer: app.run(debug=True, use_reloader=False)
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.172:5000
Press CTRL+C to quit
192.168.1.172 - - [22/Aug/2025 16:40:38] "GET / HTTP/1.1" 200 -
192.168.1.172 - - [22/Aug/2025 16:40:38] "GET /static/styles.css HTTP/1.1" 200 -
192.168.1.172 - - [22/Aug/2025 16:40:38] "GET /favicon.ico HTTP/1.1" 404 -
